<a href="https://www.kaggle.com/code/kirtimathur/indian-airlines?scriptVersionId=120173785" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date as dt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/d/kirtimathur/indian-airlines-data/Data_Train.xlsx


### Analysis from customer point of view

#### Loading the Data set and checking its size.

In [ ]:
df=pd.read_excel("/kaggle/input/d/kirtimathur/indian-airlines-data/Data_Train.xlsx")
df.shape

#### Checking top 5 rows of the data

In [ ]:
df.head()

#### Basic information about data

In [ ]:
df.info()

- There is 1 missing value in two of the columns, Route and Total_Stops 

#### Checking Null values sum for columns of the dataset.

In [ ]:
df.isnull().sum()

#### Checking the row for Route, where we see both missing values are in one row

In [ ]:
df[df["Route"].isnull()] 

- So Route and Total_Stops should be accordingly.

In [ ]:
df["Total_Stops"].mode()

In [ ]:
df["Route"].mode()

- It can be inferred that mode for both Route and Total_Stops are in accordance.

#### Filling the Null values using mode for both the columns, Route and Total_Stops.

In [ ]:
df["Total_Stops"].fillna(df["Total_Stops"].mode()[0],inplace=True)

In [ ]:
df["Route"].fillna(df["Route"].mode()[0],inplace=True)

In [ ]:
#sum of all null values of data is 0 now
df.isnull().sum().sum()

#### Checking unique values for all the columns

In [ ]:
for i in df.columns:
    print(i,"---",df[i].unique())

- For Destination column, New Delhi and Delhi imply the same airport for changing New Delhi to Delhi

In [ ]:
df["Destination"]=df["Destination"].replace("New Delhi","Delhi")
df["Destination"].unique() #checking uniques for Destination column

#### Changing data tyoe of Date_of_Journey, Dep_Time, Arrival_Time columns from object to datetime

In [ ]:
df["Date_of_Journey"]=pd.to_datetime(df["Date_of_Journey"])
df["Dep_Time"]=pd.to_datetime(df["Dep_Time"])
df["Arrival_Time"]=pd.to_datetime(df["Arrival_Time"])

In [ ]:
df.info() #datatypes changed

#### Extracting day and month from date of journey; hour and minute from departure time; hour and minute from arrival time

In [ ]:
#extracting  day 
df["day_of_journey"] = df["Date_of_Journey"].dt.day
#extracting month
df["month_of_journey"] = df["Date_of_Journey"].dt.month

#extracting  departure Hours
df["Dep_hour"] = df["Dep_Time"].dt.hour
#extracting departure Minutes
df["Dep_min"] = df["Dep_Time"].dt.minute

#extracting  arrival Hours
df["arrival_hour"] = df["Arrival_Time"].dt.hour
#extracting arrival Minutes
df["arrival_min"] = df["Arrival_Time"].dt.minute

In [ ]:
df.info()

#### Dropping Departure and Arrival time columns, and date column, as they are not usefull now.

In [ ]:
df.drop(columns=["Dep_Time","Arrival_Time","Date_of_Journey"],inplace=True)

In [ ]:
df.head(3)

#### Duration columns data should to be changed to all numeric, so converting it into minutes
- eval: takes string in the form of expression and gives value accordingly

In [ ]:
df["Duration"]=df["Duration"].str.replace("h","*60").str.replace(" ","+").str.replace("m","*1").apply(eval)

In [ ]:
df.head(3)

#### Changing Total Stops data column to numeric

In [ ]:
df.replace({"non-stop":0,"1 stop":1,"2 stops":2,"3 stops":3,"4 stops":4},inplace=True)

In [ ]:
df["Total_Stops"].unique()

 #### Highest priced Airline and route of that Airline

In [ ]:
df["Price"].max()

In [ ]:
df.groupby(["Route","Price"])["Airline"].agg(["max"]).sort_values(by="Price",ascending=False)[0:1]

 #### Lowest price airlines and the route of that Airline

In [ ]:
df["Price"].min()

In [ ]:
df.groupby(["Route","Price"])["Airline"].agg(["min"]).sort_values(by="Price",ascending=False)[0:1]

#### Month in which highest price flight was booked

In [ ]:
df.groupby(["month_of_journey"])["Price"].max().sort_values(ascending=False)[0:1]

#### Month in which lowest price flight was booked

In [ ]:
df.groupby(["month_of_journey"])["Price"].min().sort_values()[0:1]

#### Most expensive and least expensive month on the basis of price to travel

In [ ]:
sns.barplot(df["month_of_journey"],df["Price"],data=df)

#### Preference of Total stops

In [ ]:
import warnings
warnings.filterwarnings("ignore")
sns.barplot(df["Total_Stops"],df["Price"],data=df,estimator=sum);

#### The highest duration of a flight

In [ ]:
df["Duration"].max()

#### Details of maximum duration flight

In [ ]:
df[df["Duration"]==df["Duration"].max()]

#### Checking correlation between numeric data

In [ ]:
df.corr()

#### Heatmap of correlation

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(df.corr(),annot=True)

- Total stops and duration has 0.74 which means there's a strong correlation.
- Price and duration has 0.51 which is fairly strong correlation.
- Price and total stops has 0.6 which is strong correlation.

#### Statistical summary

In [ ]:
df.describe().T

- Least duration can't be 5 minutes, so have to correct it.

In [ ]:
df.describe(include="object").T

#### Checking data of duration 5 row

In [ ]:
df[df["Duration"]==5]

#### Looking for rows of similar data

In [ ]:
x=df[(df["Source"]=="Mumbai") & (df["Destination"]=="Hyderabad") & (df["Total_Stops"]==2)]
x

In [ ]:
x["Duration"].mode() 

- Have to consider all the factors like airline, route, so using mean of above data to replace the faulty data.

In [ ]:
round(x["Duration"].mean(),2)

#### Replacing the faulty duration

In [ ]:
df["Duration"].iloc[6474]=round(x["Duration"].mean(),2)

In [ ]:
df.iloc[6474]

#### Univariate analysis on price

In [ ]:
sns.boxplot(df["Price"])

- Most flights had price in range 5k to 15k
- 50% of data has price is < 8k

In [ ]:
sns.distplot(df["Price"])

#### Total flights of each airlines

In [ ]:
df["Airline"].value_counts()

In [ ]:
sns.countplot("Airline",data=df)
plt.xticks(rotation=90);

#### Boxplot to see distribution of prices for all airlines

In [ ]:
sns.boxplot(x="Airline",y="Price",data=df)
plt.xticks(rotation=90)
plt.title("Prices for different airlines");

- Jet airways business is the most expensive.
- Trujet airways busniess is the least expensive.

#### Fetching details of the most expensive flight

In [ ]:
df[df["Airline"]=="Jet Airways Business"]

#### Details of the least expensive flight

In [ ]:
df[df["Airline"]=="Trujet"]

#### Total number of flights with respect to stopages

In [ ]:
df["Total_Stops"].value_counts()

#### Flights corresponding to all airline with respect to the number of stopages

In [ ]:
df.groupby(["Total_Stops"])["Airline"].value_counts()

- Indigo has highest number of flights i.e. 1241 and 0 stops

In [ ]:
sns.barplot(x="Airline",y="Total_Stops",data=df)
plt.xticks(rotation=90);

In [ ]:
plt.figure(figsize=(20,5))
plt.subplot(1,3,1)
sns.barplot(x="Airline",y="Total_Stops",data=df)
plt.xticks(rotation=90);
plt.subplot(1,3,2)
sns.barplot(x="Airline",y="Total_Stops",data=df,estimator=max)
plt.xticks(rotation=90);
plt.subplot(1,3,3)
sns.barplot(x="Airline",y="Total_Stops",data=df,estimator=min)
plt.xticks(rotation=90);

- Only vistara premium economy has 0 stops

#### Relation of price with the total stops

In [ ]:
sns.scatterplot(x="Total_Stops",y="Price",data=df)

In [ ]:
sns.boxplot(x="Total_Stops",y="Price",data=df)
plt.xticks(rotation=90)
plt.title("Price w.r.t. stops");

#### Count of flight available from source to destination

In [ ]:
df.groupby(["Source","Destination"])["Airline"].value_counts()

#### Max,min and avg price of different flights from source to destination

In [ ]:
k=df.groupby(["Source","Destination","Airline"])["Price"].agg(["mean","max","min"])
k.reset_index(inplace=True)
k

In [ ]:
k1=k[0:9]
plt.figure(figsize=(20,5))
plt.subplot(1,3,1)
sns.barplot(x="Airline",y="mean",data=k1)
plt.xticks(rotation=90);
plt.subplot(1,3,2)
sns.barplot(x="Airline",y="max",data=k1)
plt.xticks(rotation=90);
plt.subplot(1,3,3)
sns.barplot(x="Airline",y="min",data=k1)
plt.xticks(rotation=90);

#### Variation of flight price according to different sources

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x="Source",y="Price",data=df)
plt.xticks(rotation=90,size=15)
plt.title("Variation of Flight Price by Source");

#### Variation of flight price according to different destinations

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x="Destination",y="Price",data=df)
plt.xticks(rotation=90,size=15)
plt.title("Variation of Flight Price by Destination");

- Most landings are in Banglore.
- Least landings are in Kolkata.

#### Month of peak booking in terms the total number of flights took off (not from selling point of view)

In [ ]:
sns.countplot(x="month_of_journey",data=df)

- Maximum flights took off on month 6, June
- Minimum flights took of on month 4, April (off-season)

#### Peak business month based on total sales

In [ ]:
sns.barplot(x="month_of_journey",y="Price",data=df,estimator=sum) 

- June and April had maximum and minimum sale in terms of business as well on the basis of total sale.

#### Peak business month based on average sales

In [ ]:
sns.barplot(x="month_of_journey",y="Price",data=df);

- January has maximum sales based on average.

#### Month wise took off of flights 

In [ ]:
plt.scatter(df["Airline"],df["month_of_journey"])
plt.xticks(rotation=90)
plt.xlabel("airlines")
plt.ylabel("month")
plt.title("month v/s flights");

- From Indigo to Air Asia took off in all months.
- Trujet took off the least,in just one month.

#### Flights that took off in each month for each airline

In [ ]:
df.groupby(["month_of_journey"])["Airline"].value_counts()

- In January most flights of jet airways took off.

#### Variation of price with months

In [ ]:
sns.boxplot(x="month_of_journey",y="Price",data=df)
plt.xticks(rotation=90)
plt.xlabel("month")
plt.ylabel("price")
plt.title("month v/s price");

- May, June, September, December have same prices.
- Only April has less prices.

#### The peak day of flights bookings

In [ ]:
sns.countplot(x="day_of_journey",data=df);

#### Peak day of sales

In [ ]:
sns.barplot(x="day_of_journey",y="Price",data=df,estimator=sum)
plt.xticks(rotation=90)
plt.xlabel("day")
plt.ylabel("price")
plt.title("peak day")

#### Variation of day with the price

In [ ]:
sns.boxplot(x="day_of_journey",y="Price",data=df)
plt.xticks(rotation=90)
plt.xlabel("day")
plt.ylabel("price")
plt.title("day v/s price");

- Day does not matter for the prices of flights.

#### Peak departure time

In [ ]:
df["Dep_hour"].value_counts()

- 9 am is peak departure time for all airlines.

#### Peak departure time for different flights.

In [ ]:
df.groupby(["Airline"])["Dep_hour"].value_counts()

#### Peak departure hour for particular source (to find rush hour)

In [ ]:
df.groupby(["Source","Airline"])["Dep_hour"].value_counts()

#### Peak arrival hour 

In [ ]:
sns.countplot(x="arrival_hour",data=df);

- 7 PM is the peak arrival hour.

#### Peak arrival time for different flights

In [ ]:
df.groupby(["Airline"])["arrival_hour"].value_counts()

#### Peak arrival hour for particular destinations (to find rush hour)

In [ ]:
df.groupby(["Destination","Airline"])["arrival_hour"].value_counts()